In [ ]:
# Cell 0: Imports
import os
import json
import base64
import uuid
import pandas as pd
import re
from io import BytesIO
from datetime import datetime
from pyspark.sql import functions as F
from pyspark.sql.functions import lit, current_timestamp, col
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, ArrayType, TimestampType, BooleanType
from delta.tables import DeltaTable

from clinical_data_standards_framework.utils import (
    save_with_audit
)

# Note: This notebook uses mapInPandas for scalable parallel processing of SOA sheets
# Only parses 4 columns: Visit, Visit Number, Timepoint, Timepoint number


In [ ]:
# Cell 1: Check Upstream Status & Documents Found
# Check if extract_excel_sheets found documents

print("Checking upstream task status (extract_excel_sheets)...")
try:
    upstream_status = dbutils.jobs.taskValues.get(
        taskKey="extract_excel_sheets", 
        key="excel_extraction_status"
    )
    print(f"  Upstream status (extract_excel_sheets): {upstream_status}")
    
    if upstream_status == "FAILED":
        print("\n⚠️  Upstream task (extract_excel_sheets) FAILED")
        print("   Exiting...")
        dbutils.notebook.exit("SKIPPED - Upstream task failed")
    elif upstream_status == "NO_DOCUMENTS":
        print("\n⚠️  No documents to process")
        dbutils.notebook.exit("SKIPPED - No documents found")
    elif upstream_status == "SUCCESS":
        doc_count = dbutils.jobs.taskValues.get(taskKey="extract_excel_sheets", key="excel_sheets_count")
        print(f"  Upstream extracted: {doc_count} sheets")
    
except Exception as e:
    print(f"⚠ Could not read upstream status: {e}")
    print("  Continuing anyway (backwards compatibility)")

# Check if documents were found
try:
    documents_found = dbutils.jobs.taskValues.get(taskKey="extract_excel_sheets", key="documents_found")
    
    if documents_found == "false":
        print("\n⏭️  No documents were found in previous task (extract_excel_sheets)")
        print("   Skipping visits/activities processing.")
        dbutils.notebook.exit("SKIPPED - no documents to process")
    
    print(f"✓ Documents found in previous task: {documents_found}")
    
except Exception as e:
    print(f"⚠ Could not read 'documents_found' task value: {e}")
    print("  Continuing anyway (backwards compatibility)")
    pass

print("\n✓ All upstream checks passed. Proceeding with visits/activities processing...")


In [ ]:
# Cell 2: Get Configuration from Setup Task
globals_dict = json.loads(dbutils.jobs.taskValues.get(taskKey="setup", key="globals"))
pipeline_config = json.loads(dbutils.jobs.taskValues.get(taskKey="setup", key="pipeline_config"))
created_by_principal = dbutils.jobs.taskValues.get(taskKey="setup", key="created_by_principal")
databricks_job_id = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_job_id")
databricks_job_name = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_job_name")
databricks_run_id = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_run_id")

# Extract needed values from globals
catalog = globals_dict['catalog']
bronze_schema = globals_dict['bronze_schema']
silver_schema = globals_dict['silver_schema']

print(f"Catalog: {catalog}")
print(f"Bronze Schema: {bronze_schema}")
print(f"Silver Schema: {silver_schema}")
print(f"Created by: {created_by_principal}")
print(f"Job ID: {databricks_job_id}, Job Name: {databricks_job_name}, Run ID: {databricks_run_id}")


In [ ]:
# Cell 3: Construct Table Names
sheets_table = f"{catalog}.{bronze_schema}.md_dta_excel_file_raw"
history_table = f"{catalog}.{bronze_schema}.md_file_history"

# Output tables (auto-created by save_with_audit)
soa_table = f"{catalog}.{bronze_schema}.md_schedule_of_activities"
protocol_visit_table = f"{catalog}.{bronze_schema}.md_protocol_visit"
protocol_activity_table = f"{catalog}.{bronze_schema}.md_protocol_activity"
activity_visit_map_table = f"{catalog}.{bronze_schema}.md_protocol_activity_visit_map"
vendor_visit_table = f"{catalog}.{silver_schema}.md_vendor_visit_draft"  # Draft tables go to silver_schema

print(f"Sheets table: {sheets_table}")
print(f"History table: {history_table}")
print(f"Output tables:")
print(f"  SOA: {soa_table}")
print(f"  Protocol Visit: {protocol_visit_table}")
print(f"  Protocol Activity: {protocol_activity_table}")
print(f"  Activity-Visit Map: {activity_visit_map_table}")
print(f"  Vendor Visit: {vendor_visit_table}")


In [ ]:
# Cell 4: Read SOA Sheets
# Filter for sheets categorized as 'visits_and_timepoints' (from categorize_sheet_name function)

print(f"{'='*80}")
print(f"Reading SOA sheets...")
print(f"{'='*80}")

df_sheets = spark.table(sheets_table).filter(
    F.col("sheet_category") == "visits_and_timepoints"
).select(
    "document_id",
    "parent_document_id",
    "sheet_name",
    "source_file_path",
    "trial_id",
    "data_stream_type",
    "data_provider_name"
)

# Join with file history to get content_base64
df_with_content = df_sheets.join(
    spark.table(history_table).filter(
        F.col("is_current") == True
    ).select(
        F.col("extracted_path").alias("file_path"),
        "content_base64"
    ),
    df_sheets.source_file_path == F.col("file_path"),
    "left"
)

sheet_count = df_with_content.count()
print(f"Found {sheet_count} SOA sheets to process")
print(f"{'='*80}")

if sheet_count == 0:
    print("No SOA sheets to process. Exiting.")
    dbutils.notebook.exit("No SOA sheets to process")


In [ ]:
# Cell 5: Define Vendor Visit Schema

# Vendor Visit Schema (DTA-level with definition_hash)
vendor_visit_schema = StructType([
    StructField("vendor_visit_id", StringType(), False),
    StructField("protocol_visit_id", StringType(), True),
    StructField("soa_id", StringType(), True),
    StructField("parent_document_id", StringType(), True),
    StructField("dta_id", StringType(), True),
    StructField("trial_id", StringType(), False),
    StructField("data_stream_type", StringType(), False),
    StructField("data_provider_name", StringType(), False),
    StructField("visit_name", StringType(), False),
    StructField("visit_number", IntegerType(), True),
    StructField("timepoint", StringType(), True),
    StructField("timepoint_number", FloatType(), True),
    StructField("status", StringType(), True),
    StructField("notes", StringType(), True),
    StructField("processing_status", StringType(), True),
    StructField("version", StringType(), True),
    StructField("version_status", StringType(), True),
    StructField("is_current_draft", BooleanType(), True)
])

print("✓ Schema defined")


In [ ]:
# Cell 6: Helper Functions

def find_header_row(df_sheet, target_columns):
    """
    Find header row containing Visit AND Visit Number columns.
    Timepoint columns are optional but will be captured if present.
    Scans first 20 rows for a row matching the minimum required columns.
    
    Args:
        df_sheet: pandas DataFrame with headers=None
        target_columns: list of column name patterns to find
    
    Returns:
        tuple: (header_row_index, column_map) or (None, None) if not found
               column_map is {standardized_name: column_index}
    """
    max_rows = min(20, len(df_sheet))
    
    for row_idx in range(max_rows):
        row_values = df_sheet.iloc[row_idx].values
        row_str = ' '.join([str(v).lower().strip() for v in row_values if pd.notna(v)])
        
        # Check if row contains target columns
        col_map = {}
        for col_idx, val in enumerate(row_values):
            if pd.isna(val):
                continue
            
            val_lower = str(val).lower().strip()
            
            # Match target columns (flexible matching with underscores or spaces)
            # Visit or visit_name
            if ('visit' in val_lower and 'number' not in val_lower and 
                'Visit' not in col_map):
                col_map['Visit'] = col_idx
            # Visit Number or visit_number or visitnum
            elif (('visit' in val_lower and 'number' in val_lower) or 
                  val_lower == 'visit_number' or val_lower == 'visitnum') and \
                  'Visit Number' not in col_map:
                col_map['Visit Number'] = col_idx
            # Timepoint (not timepoint_number) - OPTIONAL
            elif ('timepoint' in val_lower and 'number' not in val_lower and 
                  'Timepoint' not in col_map):
                col_map['Timepoint'] = col_idx
            # Timepoint Number or timepoint_number - OPTIONAL
            elif (('timepoint' in val_lower and 'number' in val_lower) or 
                  val_lower == 'timepoint_number') and \
                  'Timepoint Number' not in col_map:
                col_map['Timepoint Number'] = col_idx
        
        # Require BOTH Visit AND Visit Number columns (minimum)
        # Timepoint columns are optional but will be captured if present
        if 'Visit' in col_map and 'Visit Number' in col_map:
            print(f"  ✓ Found valid header at row {row_idx}: {list(col_map.keys())}")
            return row_idx, col_map
    
    return None, None


def parse_timepoint_number(timepoint_str):
    """
    Extract numeric value from timepoint string.
    Examples: 'Day 1' -> 1.0, 'Week 4' -> 28.0, 'Month 3' -> 90.0
    """
    if pd.isna(timepoint_str):
        return None
    
    timepoint_str = str(timepoint_str).strip()
    
    # Extract number from string
    match = re.search(r'(\d+\.?\d*)', timepoint_str)
    if not match:
        return None
    
    num = float(match.group(1))
    
    # Convert to days based on unit
    timepoint_lower = timepoint_str.lower()
    if 'week' in timepoint_lower:
        return num * 7
    elif 'month' in timepoint_lower:
        return num * 30
    elif 'year' in timepoint_lower:
        return num * 365
    else:
        return num


def is_instruction_row(visit_val, visit_num_val):
    """
    Detect if a row is an instruction/description row rather than actual data.
    
    Instruction rows typically have:
    - Long text descriptions (> 50 chars)
    - Keywords like "specify", "applicable", "provide", "leave blank"
    - Non-numeric or missing visit numbers
    
    Args:
        visit_val: Value from Visit column
        visit_num_val: Value from Visit Number column
    
    Returns:
        True if row appears to be instructions, False if it's data
    """
    if pd.isna(visit_val):
        return True
    
    visit_str = str(visit_val).strip().lower()
    
    # Check for instruction keywords
    instruction_keywords = [
        'specify', 'applicable', 'otherwise', 'leave blank',
        'provide', 'external data provider', 'transfer values',
        'if applicable', 'additional info', 'note:', 'instructions:',
        'specify transfer', 'as provided by', 'color legend'
    ]
    
    if any(keyword in visit_str for keyword in instruction_keywords):
        return True
    
    # Check if visit name is too long (likely a description)
    if len(visit_str) > 50:
        return True
    
    # Check if visit_number looks like instructions (not numeric-ish)
    if pd.notna(visit_num_val):
        visit_num_str = str(visit_num_val).strip().lower()
        # If visit number contains instruction words, it's likely an instruction row
        if any(keyword in visit_num_str for keyword in instruction_keywords):
            return True
    
    return False

print("✓ Helper functions defined")


In [ ]:
# Cell 7: Process SOA Sheets Using mapInPandas
# Scalable parallel processing across Spark workers

print(f"{'='*80}")
print(f"Processing SOA sheets with mapInPandas...")
print(f"{'='*80}")

def process_soa_sheet(iterator):
    """
    Process SOA sheets in parallel using mapInPandas.
    Parses only 4 columns: Visit, Visit Number, Timepoint, Timepoint Number.
    Auto-generates protocol-level data where not available.
    """
    results = []
    
    for pdf in iterator:
        for idx, row in pdf.iterrows():
            document_id = row['document_id']
            parent_document_id = row['parent_document_id']
            sheet_name = row['sheet_name']
            source_path = row['source_file_path']
            content_b64 = row['content_base64']
            trial_id = row['trial_id']
            data_stream_type = row['data_stream_type']
            data_provider_name = row['data_provider_name']
            
            # Check for missing content
            if pd.isna(content_b64) or not content_b64:
                vendor_visit_id = str(uuid.uuid4())
                results.append({
                    'vendor_visit_id': vendor_visit_id,
                    'protocol_visit_id': None,
                    'soa_id': None,
                    'parent_document_id': parent_document_id,
                    'dta_id': None,
                    'trial_id': trial_id,
                    'data_stream_type': data_stream_type,
                    'data_provider_name': data_provider_name,
                    'visit_name': f"SHEET:{sheet_name}",
                    'visit_number': None,
                    'timepoint': None,
                    'timepoint_number': None,
                    'status': 'MANUAL_REVIEW',
                    'notes': 'No Excel content available (content_base64 is null)',
                    'processing_status': 'ERROR',
                    'version': None,
                    'version_status': 'DRAFT',
                    'is_current_draft': True
                })
                continue
            
            try:
                # Decode Excel from base64
                excel_bytes = base64.b64decode(content_b64)
                file_ext = os.path.splitext(source_path.lower())[1]
                engine = 'xlrd' if file_ext in ['.xls', '.xlt'] else 'openpyxl'
                
                # Read sheet without headers
                df_sheet = pd.read_excel(BytesIO(excel_bytes), sheet_name=sheet_name, header=None, dtype=str)
                
                # Find header row
                header_row, col_map = find_header_row(df_sheet, ['Visit', 'Visit Number', 'Timepoint', 'Timepoint Number'])
                
                if header_row is None:
                    vendor_visit_id = str(uuid.uuid4())
                    results.append({
                        'vendor_visit_id': vendor_visit_id,
                        'protocol_visit_id': None,
                        'soa_id': None,
                        'parent_document_id': parent_document_id,
                        'dta_id': None,
                        'trial_id': trial_id,
                        'data_stream_type': data_stream_type,
                        'data_provider_name': data_provider_name,
                        'visit_name': f"SHEET:{sheet_name}",
                        'visit_number': None,
                        'timepoint': None,
                        'timepoint_number': None,
                        'status': 'MANUAL_REVIEW',
                        'notes': 'Header detection failed: Could not find Visit column',
                        'processing_status': 'ERROR',
                        'version': None,
                        'version_status': 'DRAFT',
                        'is_current_draft': True
                    })
                    continue
                
                # Generate SOA ID
                soa_id = str(uuid.uuid4())
                
                # Process data rows (after header)
                for row_idx in range(header_row + 1, len(df_sheet)):
                    row_values = df_sheet.iloc[row_idx].values
                    
                    # Skip empty rows
                    non_empty = [v for v in row_values if pd.notna(v) and str(v).strip()]
                    if len(non_empty) < 1:
                        continue
                    
                    # Extract required columns
                    visit_name = None
                    visit_number = None
                    timepoint = None
                    timepoint_number = None
                    
                    # Get raw values first for instruction detection
                    if 'Visit' in col_map:
                        visit_val = row_values[col_map['Visit']]
                        if pd.notna(visit_val):
                            visit_name = str(visit_val).strip()
                    
                    if 'Visit Number' in col_map:
                        visit_num_val = row_values[col_map['Visit Number']]
                    else:
                        visit_num_val = None
                    
                    # ========================================
                    # SKIP INSTRUCTION ROWS
                    # Check if this row contains instructions/descriptions rather than actual data
                    # ========================================
                    if is_instruction_row(visit_name, visit_num_val):
                        continue  # Skip this row, it's instructions not data
                    
                    # Parse visit number (only for data rows)
                    if pd.notna(visit_num_val):
                        try:
                            visit_number = int(float(str(visit_num_val).strip()))
                        except:
                            pass
                    
                    if 'Timepoint' in col_map:
                        tp_val = row_values[col_map['Timepoint']]
                        if pd.notna(tp_val):
                            timepoint = str(tp_val).strip()
                    
                    if 'Timepoint Number' in col_map:
                        tp_num_val = row_values[col_map['Timepoint Number']]
                        if pd.notna(tp_num_val):
                            try:
                                timepoint_number = float(str(tp_num_val).strip())
                            except:
                                # Try to parse from timepoint string
                                if timepoint:
                                    timepoint_number = parse_timepoint_number(timepoint)
                    
                    # Auto-generate timepoint_number if not provided but timepoint exists
                    if timepoint and not timepoint_number:
                        timepoint_number = parse_timepoint_number(timepoint)
                    
                    # Skip row if no visit name
                    if not visit_name:
                        continue
                    
                    # Create vendor visit record
                    vendor_visit_id = str(uuid.uuid4())
                    protocol_visit_id = str(uuid.uuid4())  # Will be matched to protocol visit later
                    
                    results.append({
                        'vendor_visit_id': vendor_visit_id,
                        'protocol_visit_id': protocol_visit_id,
                        'soa_id': soa_id,
                        'parent_document_id': parent_document_id,
                        'dta_id': None,
                        'trial_id': trial_id,
                        'data_stream_type': data_stream_type,
                        'data_provider_name': data_provider_name,
                        'visit_name': visit_name,
                        'visit_number': visit_number,
                        'timepoint': timepoint,
                        'timepoint_number': timepoint_number,
                        'status': 'ACTIVE',
                        'notes': None,
                        'processing_status': 'SUCCESS',
                        'version': None,
                        'version_status': 'DRAFT',
                        'is_current_draft': True
                    })
            
            except Exception as e:
                vendor_visit_id = str(uuid.uuid4())
                results.append({
                    'vendor_visit_id': vendor_visit_id,
                    'protocol_visit_id': None,
                    'soa_id': None,
                    'parent_document_id': parent_document_id,
                    'dta_id': None,
                    'trial_id': trial_id,
                    'data_stream_type': data_stream_type,
                    'data_provider_name': data_provider_name,
                    'visit_name': f"SHEET:{sheet_name}",
                    'visit_number': None,
                    'timepoint': None,
                    'timepoint_number': None,
                    'status': 'MANUAL_REVIEW',
                    'notes': f"Processing error: {str(e)}",
                    'processing_status': 'ERROR',
                    'version': None,
                    'version_status': 'DRAFT',
                    'is_current_draft': True
                })
    
    # Return results as pandas DataFrame
    yield pd.DataFrame(results)

# Process sheets in parallel
df_vendor_visits = df_with_content.mapInPandas(process_soa_sheet, vendor_visit_schema)

vendor_visit_count = df_vendor_visits.count()
print(f"\n✓ Processed {vendor_visit_count} vendor visits")
print(f"{'='*80}")


In [ ]:
# Cell 8: Save Vendor Visits to Silver
# Only vendor visits are saved as they are the main library entity
# Protocol-level tables (SOA, Protocol Visit, Protocol Activity) will be generated during approval

print(f"{'='*80}")
print(f"Saving vendor visits to silver...")
print(f"{'='*80}")

if vendor_visit_count > 0:
    # Save using save_with_audit (auto-creates table if needed)
    save_with_audit(
        df=df_vendor_visits,
        table_name=vendor_visit_table,
        created_by_principal=created_by_principal,
        databricks_job_id=databricks_job_id,
        databricks_job_name=databricks_job_name,
        databricks_run_id=databricks_run_id,
        mode="append"
    )
    
    print(f"✓ Saved {vendor_visit_count} vendor visits to {vendor_visit_table}")
else:
    print("⚠ No vendor visits to save")

print(f"{'='*80}")


In [ ]:
# Cell 9: Summary
print(f"{'='*80}")
print(f"VISITS & ACTIVITIES PROCESSING SUMMARY")
print(f"{'='*80}")
print(f"Vendor Visits Processed: {vendor_visit_count}")
print(f"\nOutput Table: {vendor_visit_table}")
print(f"\nNote: Protocol-level tables (SOA, Protocol Visit, Protocol Activity) will be")
print(f"      generated during DTA approval when vendor visits are promoted to gold.")
print(f"{'='*80}")

# Set task values for downstream tasks
dbutils.jobs.taskValues.set(key="vendor_visits_count", value=str(vendor_visit_count))
dbutils.jobs.taskValues.set(key="visits_processing_status", value="SUCCESS" if vendor_visit_count > 0 else "NO_DATA")
